In [1]:
import pandas as pd

### 1. 데이터 로드

In [39]:
ad = pd.read_csv(r'data-files\03-ad.csv')
log = pd.read_csv(r'data-files\03-log.csv')
od = pd.read_csv(r'data-files\03-order.csv')

In [40]:
print('='*70)
display(ad.head(3))
print('='*70)
display(log.head(3))
print('='*70)
display(od.head(3))

,date,campaign,ad_group,creative,spend,clicks
0,2023-01-01,Campaign_1,AdGroup_1,Creative_1,237.659902,22
1,2023-01-01,Campaign_1,AdGroup_1,Creative_2,469.650812,19
2,2023-01-01,Campaign_1,AdGroup_1,Creative_3,186.049658,89


,order_id,campaign,timestamp,event
0,1174,Campaign_3,2023-02-23 18:15:58,purchase
1,1491,NaN,2023-02-17 13:19:47,purchase
2,1163,Campaign_2,2023-01-08 09:05:28,purchase


,order_id,order_date,order_amount
0,1,2023-02-07,663.424911
1,2,2023-02-13,1373.031257
2,3,2023-01-13,714.409280


In [41]:
# 자료형 변환
ad['date'] = pd.to_datetime(ad['date'])
log['timestamp'] = pd.to_datetime(log['timestamp'])
od['order_date'] = pd.to_datetime(od['order_date'])

### 2. 주간 성과 확인

(1) CPC 계산

In [42]:
# 주간 성과 집계가 목적이므로 week 컬럼 추가
ad['week'] = ad['date'].dt.to_period('W').dt.to_timestamp()

# 주간 광고 캠페인별 CPC 계산
ad_agg = ad.groupby(by=['campaign', 'week'])[['spend', 'clicks']].sum().reset_index()
ad_agg['cpc'] = ad_agg['spend']/ad_agg['clicks']

# 집계 결과 출력
ad_agg.head()

,campaign,week,spend,clicks,cpc
0,Campaign_1,2022-12-26,2241.418708,313,7.161082
1,Campaign_1,2023-01-02,14602.644897,3517,4.152017
2,Campaign_1,2023-01-09,18568.019047,3389,5.478908
3,Campaign_1,2023-01-16,16614.755062,3677,4.518563
4,Campaign_1,2023-01-23,16701.262833,3384,4.935361


(2) ROAS 주간 ROAS 계산

In [70]:
# 구매 데이터에 로그 데이터를 결합
od1 = pd.merge(od, log
               , left_on = 'order_id', right_on = 'order_id'
               , how = 'left'
)

# 캠페인을 통한 로그가 남지 않은 경우를 organic으로 지정
od1['campaign'] = od1['campaign'].fillna('organic')

# 주간 판매 성과 집계
od1['week'] = od['order_date'].dt.to_period('W').dt.to_timestamp()
od_agg = od1.groupby(by=['campaign', 'week'])['order_amount'].sum().reset_index()

# 집계 결과 출력
od_agg.head()

,campaign,week,order_amount
0,Campaign_1,2022-12-26,5888.163592
1,Campaign_1,2023-01-02,29067.678837
2,Campaign_1,2023-01-09,27335.893357
3,Campaign_1,2023-01-16,23543.017742
4,Campaign_1,2023-01-23,24237.367534


In [75]:
# 집계 결과 결합
df = pd.merge(od_agg, ad_agg
              , how = 'left'
              , left_on = ['campaign', 'week']
              , right_on = ['campaign', 'week']
)

# ROAS 계산
df['roas'] = df['order_amount'] / df['spend']
df.head()

,campaign,week,order_amount,spend,clicks,cpc,roas
0,Campaign_1,2022-12-26,5888.163592,2241.418708,313.0,7.161082,2.626981
1,Campaign_1,2023-01-02,29067.678837,14602.644897,3517.0,4.152017,1.990576
2,Campaign_1,2023-01-09,27335.893357,18568.019047,3389.0,5.478908,1.472203
3,Campaign_1,2023-01-16,23543.017742,16614.755062,3677.0,4.518563,1.416995
4,Campaign_1,2023-01-23,24237.367534,16701.262833,3384.0,4.935361,1.451230


(3) 주별 ROAS 변화 추이 확인

In [80]:
# pivot_table로 reshaping
df.pivot_table(
    index='campaign'
    , columns='week'
    , values='roas'
)

week,2022-12-26,2023-01-02,2023-01-09,2023-01-16,2023-01-23,2023-01-30,2023-02-06,2023-02-13,2023-02-20,2023-02-27
campaign,,,,,,,,,,
Campaign_1,2.626981,1.990576,1.472203,1.416995,1.451230,1.809280,1.190942,1.118135,1.579390,1.217456
Campaign_2,0.850365,1.180272,1.377531,1.607622,1.483889,2.071922,2.053962,1.044400,1.482987,1.098304
Campaign_3,0.620534,0.689514,0.512379,0.352103,0.548467,0.756456,0.730515,0.513122,0.905600,1.206279
